In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

url = "https://www.solodeportes.com.ar/?gad_campaignid=6514863751"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
time.sleep(2)

def buscar(producto):
    try:
        barra_busqueda = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "search"))
        )
        barra_busqueda.clear()
        barra_busqueda.send_keys(producto)
        barra_busqueda.send_keys(Keys.ENTER)
        time.sleep(2)
    except:
        driver.get(url)
        time.sleep(2)
        barra_busqueda = driver.find_element(By.ID, "search")
        barra_busqueda.clear()
        barra_busqueda.send_keys(producto)
        barra_busqueda.send_keys(Keys.ENTER)
        time.sleep(2)
    
def extraer_links(cantidad):
    listado_url = []
    
    try:
        listado_productos = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ol.products.list.items.product-items"))
        )
        li_items = listado_productos.find_elements(By.CSS_SELECTOR, "li.item.product.product-item")
        
        for li in li_items[:cantidad]:
            try:
                link = li.find_element(By.CSS_SELECTOR, "div.product-item-info a")
                href = link.get_attribute("href")
                listado_url.append(href)
            except:
                continue
                
    except:
        li_items = driver.find_elements(By.CSS_SELECTOR, "li.item.product.product-item")
        for li in li_items[:cantidad]:
            try:
                link = li.find_element(By.CSS_SELECTOR, "a")
                href = link.get_attribute("href")
                listado_url.append(href)
            except:
                continue
        
    return listado_url    

def explorar_links(lista):
    datos = []
    
    for link in lista:
        try:
            driver.get(link)
            time.sleep(1)
            
            descripcion = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "value p"))
            ).text
            
            precio = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "price"))
            ).text
            
            datos.append({"Descripción": descripcion, "Precio": precio})
        except:
            try:
                descripcion = driver.find_element(By.CLASS_NAME, "value p").text
                precio = driver.find_element(By.CLASS_NAME, "price").text
                datos.append({"Descripción": descripcion, "Precio": precio})
            except:
                continue
            print("No encontrado")
    
    return datos

def main():
    total_categorias = []
        
    categorias = {
        "zapatillas adidas": 10,
        "campera nike": 7,
        "botines puma": 5,
        "remera mujer": 10
    }
    
    for categoria, cantidad in categorias.items():
        buscar(categoria)
        listado_url = extraer_links(cantidad)
        data = explorar_links(listado_url)
        total_categorias.extend(data)
        
        driver.get(url)
        time.sleep(1)

    df = pd.DataFrame(total_categorias)
    df.to_csv("new_Data.csv", index=False, encoding='utf-8')
    
    driver.quit()
    return df

if __name__ == "__main__":
    main()